In [1]:
import os
import gc
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from joblib import load, dump
from tqdm import tqdm
from scipy.signal import butter, lfilter, medfilt


# Constants

In [2]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_1 = 'gait-dbase-1/'

N_SAMPLES = 1024
FORCE_THRESHOLD = 20
TOE_HEIGTH_THRESHOLD = 40

# Filter 

In [3]:
class LowPassFilter(object):
    @staticmethod
    def butter_lowpass(cutoff, fs, order):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def apply(data, cutoff=6, fs=1500, order=2, axis=-1):
        b, a = LowPassFilter.butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data, axis=axis)
        y = y - np.median(y)
        return y


# Functions

In [4]:
def standardize(x):
    # return (x - np.mean(x, axis=0)) / np.std(x, axis=0)
    return (x - np.min(x, axis=0)) / (np.max(x, axis=0) - np.min(x, axis=0))

def get_processed_grf(grf, clip=False):
    singn_corrected = grf # if np.mean(grf) > 20 else (-grf)
    processed_grf = LowPassFilter.apply(singn_corrected, fs=1500, cutoff=15)
    processed_grf = processed_grf - np.median(processed_grf)
    if clip:
        processed_grf[processed_grf < FORCE_THRESHOLD] = 0
    return resample(processed_grf, N_SAMPLES * 3, axis=0).T

def get_processed_moment(moment):
    processed_moment = LowPassFilter.apply(moment, fs=1500, cutoff=20)
    return resample(processed_moment, N_SAMPLES * 3, axis=0).T

def get_trajectories(content, foot='right'):
    if foot == 'right':
        trajectory_x = content['data']['points'][0, 9:15, :]
        trajectory_y = content['data']['points'][1, 9:15, :]
        trajectory_z = content['data']['points'][2, 9:15, :]

    elif foot == 'left':
        trajectory_x = content['data']['points'][0, 20:26, :]
        trajectory_y = content['data']['points'][1, 20:26, :]
        trajectory_z = content['data']['points'][2, 20:26, :]

    else:
        raise ValueError('Third foot is not allowed')

    trajectory_x = LowPassFilter.apply(trajectory_x, fs=100, cutoff=6, order=4)
    trajectory_y = LowPassFilter.apply(trajectory_y, fs=100, cutoff=6, order=4)
    trajectory_z = LowPassFilter.apply(trajectory_z, fs=100, cutoff=6, order=4)

    trajectory_x = resample(trajectory_x, N_SAMPLES * 3, axis=1).T
    trajectory_y = resample(trajectory_y, N_SAMPLES * 3, axis=1).T
    trajectory_z = resample(trajectory_z, N_SAMPLES * 3, axis=1).T

    return trajectory_x, trajectory_y, trajectory_z

def get_force(content):
    force_x = []
    force_y = []
    force_z = []
    for i in range(2):
        force_x.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][1, :]
            )
        )
        force_y.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][0, :]
            )
        )
        force_z.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][2, :], True
            )
        )

    return force_x, force_y, force_z

def get_moment(content):
    moment_x = []
    moment_y = []
    moment_z = []
    for i in range(2):
        moment_x.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][1, :]
            )
        )
        moment_y.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][0, :]
            )
        )
        moment_z.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][2, :]
            )
        )

    return moment_x, moment_y, moment_z

# Test Bench

In [ ]:
path = '/home/ai/Python/GRF-Synthesis-from-Motion-Trajectories/Dataset/gait-dbase-1/2014001/2014001_C1_04.c3d'
content = c3d(path, extract_forceplat_data=True)

In [ ]:
trajectory_x_r, trajectory_y_r, trajectory_z_r = get_trajectories(content, 'right')
trajectory_x_l, trajectory_y_l, trajectory_z_l = get_trajectories(content, 'left')

force_x, force_y, force_z = get_force(content)
moment_x, moment_y, moment_z = get_moment(content)

In [ ]:
plt.plot(trajectory_z_l[:, 0])
plt.plot(trajectory_z_r[:, 0])
plt.show()

In [ ]:
plt.plot(force_y[0])
plt.plot(force_y[1])
plt.show()

In [ ]:
plt.plot(moment_y[0])
plt.plot(moment_y[1])
plt.show()

In [ ]:
mask_force_z_1 = (force_z[0] > FORCE_THRESHOLD)
mask_force_z_2 = (force_z[1] > FORCE_THRESHOLD)

start_1 = np.min(np.argwhere(mask_force_z_1 == True))
end_1 = np.max(np.argwhere(mask_force_z_1 == True))

start_2 = np.min(np.argwhere(mask_force_z_2 == True))
end_2 = np.max(np.argwhere(mask_force_z_2 == True))

force_x_1 = force_x[0][start_1 : end_1]
force_y_1 = force_y[0][start_1 : end_1]
force_z_1 = force_z[0][start_1 : end_1]

force_x_2 = force_x[1][start_2 : end_2]
force_y_2 = force_y[1][start_2 : end_2]
force_z_2 = force_z[1][start_2 : end_2]

temp_l = trajectory_z_l[start_1 : end_1, 0]
temp_r = trajectory_z_r[start_1 : end_1, 0]

force_x_r = 0
force_y_r = 0
force_z_r = 0

force_x_l = 0
force_y_l = 0
force_z_l = 0

if np.sum(temp_l) > np.sum(temp_r):
    force_x_r = force_x_1
    force_y_r = force_y_1
    force_z_r = force_z_1

    force_x_l = force_x_2
    force_y_l = force_y_2
    force_z_l = force_z_2

    trajectory_x_r = trajectory_x_l[start_2 : end_2]
    trajectory_y_r = trajectory_y_r[start_2 : end_2]
    trajectory_z_r = trajectory_z_r[start_2 : end_2]

    trajectory_x_l = trajectory_x_r[start_1 : end_1]
    trajectory_y_l = trajectory_y_l[start_1 : end_1]
    trajectory_z_l = trajectory_z_l[start_1 : end_1]

else:
    force_x_l = force_x_1
    force_y_l = force_y_1
    force_z_l = force_z_1

    force_x_r = force_x_2
    force_y_r = force_y_2
    force_z_r = force_z_2

    trajectory_x_r = trajectory_x_l[start_1 : end_1]
    trajectory_y_r = trajectory_y_r[start_1 : end_1]
    trajectory_z_r = trajectory_z_r[start_1 : end_1]

    trajectory_x_l = trajectory_x_r[start_2 : end_2]
    trajectory_y_l = trajectory_y_l[start_2 : end_2]
    trajectory_z_l = trajectory_z_l[start_2 : end_2]

# ... Resample
trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=0)
trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=0)
trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=0)

trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=0)
trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=0)
trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=0)

force_x_r = resample(force_x_r, N_SAMPLES, axis=0)
force_y_r = resample(force_y_r, N_SAMPLES, axis=0)
force_z_r = resample(force_z_r, N_SAMPLES, axis=0)

force_x_l = resample(force_x_l, N_SAMPLES, axis=0)
force_y_l = resample(force_y_l, N_SAMPLES, axis=0)
force_z_l = resample(force_z_l, N_SAMPLES, axis=0)

# ... Standardize
trajectory_x_r = standardize(trajectory_x_r)
trajectory_y_r = standardize(trajectory_y_r)
trajectory_z_r = standardize(trajectory_z_r)

trajectory_x_l = standardize(trajectory_x_l)
trajectory_y_l = standardize(trajectory_y_l)
trajectory_z_l = standardize(trajectory_z_l)

force_x_r = standardize(force_x_r)
force_y_r = standardize(force_y_r)
force_z_r = standardize(force_z_r)

force_x_l = standardize(force_x_l)
force_y_l = standardize(force_y_l)
force_z_l = standardize(force_z_l)

In [ ]:
plt.plot(trajectory_z_l[:, 0])
plt.plot(trajectory_z_r[:, 0])
plt.plot(force_z_l)
plt.plot(force_z_r)

# Extract Features

In [5]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
subjects = os.listdir(path)

features = pd.DataFrame()
target = pd.DataFrame()

records = []
n_samples = 0
n_bad_samples = 0

subjects = subjects[40:]

for subject in tqdm(subjects):
    #     print(subject)
    files_path = os.path.join(path, subject)
    files = os.listdir(files_path)
    for filename in files:

        if "ST" in filename:
            continue

        # if "C4" not in filename:
        #     continue

        # print(subject, filename, end=' ')

        try:
            content = c3d(
                os.path.join(path, subject, filename), extract_forceplat_data=True
            )

            # print('loaded', end=' ')

            trajectory_x_r, trajectory_y_r, trajectory_z_r = get_trajectories(
                content, "right"
            )
            trajectory_x_l, trajectory_y_l, trajectory_z_l = get_trajectories(
                content, "left"
            )

            force_x, force_y, force_z = get_force(content)
            moment_x, moment_y, moment_z = get_moment(content)

            del content
            gc.collect()

            force_combined_z = force_z[0] + force_z[1]
            force_mask = force_combined_z >= FORCE_THRESHOLD

            # # ... Remove all zero GRF samples
            # if (
            #     np.sum(force_z[0]) <= FORCE_THRESHOLD
            #     or np.sum(force_z[1]) <= FORCE_THRESHOLD
            # ):
            #     # print(os.path.join(path, subject, filename))
            #     n_bad_samples = n_bad_samples + 1
            #     continue

            # # ... Remove bad samples
            # if np.mean(force_mask) <= 0.5:
            #     # print(os.path.join(path, subject, filename))
            #     n_bad_samples = n_bad_samples + 1
            #     continue

            # # ... Remove samples that starts and ends abruptly
            # if (
            #     np.mean(force_combined_z[:10]) >= 200
            #     or np.mean(force_combined_z[-10:]) >= 200
            # ):
            #     # print(os.path.join(path, subject, filename))
            #     n_bad_samples = n_bad_samples + 1
            #     continue

            mask_force_z_1 = force_z[0] > FORCE_THRESHOLD
            mask_force_z_2 = force_z[1] > FORCE_THRESHOLD

            start_1 = np.min(np.argwhere(mask_force_z_1 == True))
            end_1 = np.max(np.argwhere(mask_force_z_1 == True))

            # if (end_1 - start_1) < 700 or (end_1 - start_1) > 1700:
            #     n_bad_samples = n_bad_samples + 1
            #     continue

            start_2 = np.min(np.argwhere(mask_force_z_2 == True))
            end_2 = np.max(np.argwhere(mask_force_z_2 == True))

            # if (end_2 - start_2) < 700 or (end_2 - start_2) > 1700:
            #     n_bad_samples = n_bad_samples + 1
            #     continue

            # print('1', end=' ')

            force_x_1 = force_x[0][start_1:end_1]
            force_y_1 = force_y[0][start_1:end_1]
            force_z_1 = force_z[0][start_1:end_1]

            force_x_2 = force_x[1][start_2:end_2]
            force_y_2 = force_y[1][start_2:end_2]
            force_z_2 = force_z[1][start_2:end_2]

            moment_x_1 = moment_x[0][start_1:end_1]
            moment_y_1 = moment_y[0][start_1:end_1]
            moment_z_1 = moment_z[0][start_1:end_1]

            moment_x_2 = moment_x[1][start_2:end_2]
            moment_y_2 = moment_y[1][start_2:end_2]
            moment_z_2 = moment_z[1][start_2:end_2]

            temp_l = trajectory_z_l[start_1:end_1, 0]
            temp_r = trajectory_z_r[start_1:end_1, 0]

            force_x_r = 0
            force_y_r = 0
            force_z_r = 0

            force_x_l = 0
            force_y_l = 0
            force_z_l = 0

            moment_x_r = 0
            moment_y_r = 0
            moment_z_r = 0

            moment_x_l = 0
            moment_y_l = 0
            moment_z_l = 0

            if np.sum(temp_l) > np.sum(temp_r):
                force_x_r = force_x_1
                force_y_r = force_y_1
                force_z_r = force_z_1

                force_x_l = force_x_2
                force_y_l = force_y_2
                force_z_l = force_z_2

                moment_x_r = moment_x_1
                moment_y_r = moment_y_1
                moment_z_r = moment_z_1

                moment_x_l = moment_x_2
                moment_y_l = moment_y_2
                moment_z_l = moment_z_2

                trajectory_x_r = trajectory_x_l[start_2:end_2]
                trajectory_y_r = trajectory_y_r[start_2:end_2]
                trajectory_z_r = trajectory_z_r[start_2:end_2]

                trajectory_x_l = trajectory_x_r[start_1:end_1]
                trajectory_y_l = trajectory_y_l[start_1:end_1]
                trajectory_z_l = trajectory_z_l[start_1:end_1]

            else:
                force_x_l = force_x_1
                force_y_l = force_y_1
                force_z_l = force_z_1

                force_x_r = force_x_2
                force_y_r = force_y_2
                force_z_r = force_z_2

                moment_x_l = moment_x_1
                moment_y_l = moment_y_1
                moment_z_l = moment_z_1

                moment_x_r = moment_x_2
                moment_y_r = moment_y_2
                moment_z_r = moment_z_2

                trajectory_x_r = trajectory_x_l[start_1:end_1]
                trajectory_y_r = trajectory_y_r[start_1:end_1]
                trajectory_z_r = trajectory_z_r[start_1:end_1]

                trajectory_x_l = trajectory_x_r[start_2:end_2]
                trajectory_y_l = trajectory_y_l[start_2:end_2]
                trajectory_z_l = trajectory_z_l[start_2:end_2]

            # print('2', end=' ')

            # ... Resample
            trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=0)
            trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=0)
            trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=0)

            trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=0)
            trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=0)
            trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=0)

            force_x_r = resample(force_x_r, N_SAMPLES, axis=0)
            force_y_r = resample(force_y_r, N_SAMPLES, axis=0)
            force_z_r = resample(force_z_r, N_SAMPLES, axis=0)

            force_x_l = resample(force_x_l, N_SAMPLES, axis=0)
            force_y_l = resample(force_y_l, N_SAMPLES, axis=0)
            force_z_l = resample(force_z_l, N_SAMPLES, axis=0)

            moment_x_r = resample(moment_x_r, N_SAMPLES, axis=0)
            moment_y_r = resample(moment_y_r, N_SAMPLES, axis=0)
            moment_z_r = resample(moment_z_r, N_SAMPLES, axis=0)

            moment_x_l = resample(moment_x_l, N_SAMPLES, axis=0)
            moment_y_l = resample(moment_y_l, N_SAMPLES, axis=0)
            moment_z_l = resample(moment_z_l, N_SAMPLES, axis=0)

            # print('3', end=' ')

            # ... sign correction
            if np.mean(force_y_r[: int(N_SAMPLES / 2)]) > 0:
                force_y_r = -force_y_r
            if np.mean(force_y_l[: int(N_SAMPLES / 2)]) > 0:
                force_y_l = -force_y_l

            if np.mean(force_x_r) < 0:
                force_x_r = -force_x_r
            if np.mean(force_x_l) < 0:
                force_x_l = -force_x_l

            if np.mean(moment_y_r) < 0:
                moment_y_r = -moment_y_r
            if np.mean(moment_y_l) < 0:
                moment_y_l = -moment_y_l

            if np.mean(moment_x_r[: int(N_SAMPLES / 2)]) > np.mean(moment_x_r[int(N_SAMPLES / 2):]):
                moment_x_r = -moment_x_r
            if np.mean(moment_x_l[: int(N_SAMPLES / 2)]) > np.mean(moment_x_l[int(N_SAMPLES / 2):]):
                moment_x_l = -moment_x_l

            if np.mean(moment_z_r[: int(N_SAMPLES / 2)]) < np.mean(moment_z_r[int(N_SAMPLES / 2):]):
                moment_z_r = -moment_z_r
            if np.mean(moment_z_l[: int(N_SAMPLES / 2)]) < np.mean(moment_z_l[int(N_SAMPLES / 2):]):
                moment_z_l = -moment_z_l

            # if np.mean(moment_y_r) < 0:
            #     moment_y_r = -moment_y_r
            # if np.mean(moment_y_l) < 0:
            #     moment_y_l = -moment_y_l

            # if np.mean(moment_z_r) < 0:
            #     moment_z_r = -moment_z_r
            # if np.mean(moment_z_l) < 0:
            #     moment_z_l = -moment_z_l

            # ... Standardize
            trajectory_x_r = standardize(trajectory_x_r)
            trajectory_y_r = standardize(trajectory_y_r)
            trajectory_z_r = standardize(trajectory_z_r)

            trajectory_x_l = standardize(trajectory_x_l)
            trajectory_y_l = standardize(trajectory_y_l)
            trajectory_z_l = standardize(trajectory_z_l)

            force_x_r = standardize(force_x_r)
            force_y_r = standardize(force_y_r)
            force_z_r = standardize(force_z_r)

            force_x_l = standardize(force_x_l)
            force_y_l = standardize(force_y_l)
            force_z_l = standardize(force_z_l)

            moment_x_r = standardize(moment_x_r)
            moment_y_r = standardize(moment_y_r)
            moment_z_r = standardize(moment_z_r)

            moment_x_l = standardize(moment_x_l)
            moment_y_l = standardize(moment_y_l)
            moment_z_l = standardize(moment_z_l)

            # print('4', end=' ')

            _features_l = pd.DataFrame(
                np.concatenate(
                    [trajectory_x_l, trajectory_y_l, trajectory_z_l], axis=1
                ),
                columns=[
                    "fal_x",
                    "tam_x",
                    "fcc_x",
                    "fm1_x",
                    "fm2_x",
                    "fm5_x",
                    "fal_y",
                    "tam_y",
                    "fcc_y",
                    "fm1_y",
                    "fm2_y",
                    "fm5_y",
                    "fal_z",
                    "tam_z",
                    "fcc_z",
                    "fm1_z",
                    "fm2_z",
                    "fm5_z",
                ],
            )

            _features_r = pd.DataFrame(
                np.concatenate(
                    [trajectory_x_r, trajectory_y_r, trajectory_z_r], axis=1
                ),
                columns=[
                    "fal_x",
                    "tam_x",
                    "fcc_x",
                    "fm1_x",
                    "fm2_x",
                    "fm5_x",
                    "fal_y",
                    "tam_y",
                    "fcc_y",
                    "fm1_y",
                    "fm2_y",
                    "fm5_y",
                    "fal_z",
                    "tam_z",
                    "fcc_z",
                    "fm1_z",
                    "fm2_z",
                    "fm5_z",
                ],
            )

            _features = pd.concat([_features_l, _features_r], axis=0)

            _features["filename"] = filename

            _target_r = pd.DataFrame(
                np.stack(
                    [
                        force_x_r,
                        force_y_r,
                        force_z_r,
                        moment_x_r,
                        moment_y_r,
                        moment_z_r,
                    ],
                    axis=1,
                ),
                columns=["fx", "fy", "fz", "mx", "my", "mz"],
            )

            _target_l = pd.DataFrame(
                np.stack(
                    [
                        force_x_l,
                        force_y_l,
                        force_z_l,
                        moment_x_l,
                        moment_y_l,
                        moment_z_l,
                    ],
                    axis=1,
                ),
                columns=["fx", "fy", "fz", "mx", "my", "mz"],
            )

            _target = pd.concat([_target_r, _target_l], axis=0)

            features = pd.concat([features, _features],
                                 axis=0, ignore_index=True)
            target = pd.concat([target, _target], axis=0, ignore_index=True)

            # print('5')

            records.append(filename)

            n_samples = n_samples + 1

        except Exception as e:
            print(f"ERROR [{subject}/{filename}]: {e}")
            n_bad_samples = n_bad_samples + 1
            pass


print("Number of good samples: ", n_samples)
print("Number of bad samples: ", n_bad_samples)


  0%|          | 0/10 [00:00<?, ?it/s]

ERROR [2015020/2015020_C1_05.c3d]: zero-size array to reduction operation minimum which has no identity


 10%|█         | 1/10 [00:03<00:33,  3.73s/it]

ERROR [2014022/2014022_C1_01.c3d]: invalid number of data points (0) specified
ERROR [2014022/2014022_C1_05.c3d]: invalid number of data points (0) specified
ERROR [2014022/2014022_C5_03.c3d]: invalid number of data points (0) specified


 30%|███       | 3/10 [00:10<00:24,  3.46s/it]

...

ERROR [2014050/2014050_C1_05.c3d]: invalid number of data points (0) specified
ERROR [2014050/2014050_C4_01.c3d]: invalid number of data points (0) specified
ERROR [2014050/2014050_C1_04.c3d]: zero-size array to reduction operation minimum which has no identity
ERROR [2014050/2014050_C2_05.c3d]: invalid number of data points (0) specified
ERROR [2014050/2014050_C4_02.c3d]: invalid number of data points (0) specified
ERROR [2014050/2014050_C1_01.c3d]: zero-size array to reduction operation minimum which has no identity
ERROR [2014050/2014050_C4_05.c3d]: invalid number of data points (0) specified


 90%|█████████ | 9/10 [00:30<00:03,  3.42s/it]

ERROR [2014050/2014050_C5_01.c3d]: invalid number of data points (0) specified
ERROR [2014019/2014019_C5_02.c3d]: invalid number of data points (0) specified
ERROR [2014019/2014019_C5_04.c3d]: zero-size array to reduction operation minimum which has no identity
ERROR [2014019/2014019_C1_05.c3d]: zero-size array to reduction operation minimum which has no identity
ERROR [2014019/2014019_C1_01.c3d]: zero-size array to reduction operation minimum which has no identity


100%|██████████| 10/10 [00:34<00:00,  3.42s/it]

ERROR [2014019/2014019_C1_04.c3d]: invalid number of data points (0) specified
Number of good samples:  197
Number of bad samples:  37


In [ ]:
features

In [ ]:
target

# Verification

In [ ]:
x = features['fal_z'].to_numpy().reshape(-1, N_SAMPLES)
for i in range(x.shape[0]):
    plt.plot(x[i, :])

In [ ]:
y = target['fy'].to_numpy().reshape(-1, N_SAMPLES)
for i in range(y.shape[0]):
    plt.plot(y[i, :])

In [ ]:
y = target['mx'].to_numpy().reshape(-1, N_SAMPLES)
for i in range(y.shape[0]):
    plt.plot(y[i, :])

In [ ]:
z = target['mz'].to_numpy().reshape(-1, N_SAMPLES)
for i in range(z.shape[0]):
    plt.plot(z[i, :])

In [6]:
features_new = features.drop(columns=['fcc_x', 'fcc_y', 'fcc_z'])
features_new = features_new[[
    'fal_x', 'tam_x', 'fm5_x', 'fm1_x', 'fm2_x',
    'fal_y', 'tam_y', 'fm5_y', 'fm1_y', 'fm2_y',
    'fal_z', 'tam_z', 'fm5_z', 'fm1_z', 'fm2_z',
    'filename'
]]
features_new

,fal_x,tam_x,fm5_x,fm1_x,fm2_x,fal_y,tam_y,fm5_y,fm1_y,fm2_y,fal_z,tam_z,fm5_z,fm1_z,fm2_z,filename
0,0.083794,0.074850,0.106986,0.113393,0.105180,0.096550,0.332412,0.088857,0.000000,0.072615,0.167742,0.159771,0.142265,0.312730,0.000000,2015020_C1_02.c3d
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.286422,0.000000,0.008086,0.000000,0.000000,0.000000,0.000000,0.412525,0.038397,2015020_C1_02.c3d
2,0.098673,0.095194,0.105627,0.106799,0.104145,0.115695,0.374632,0.115027,0.108928,0.112300,0.087820,0.097543,0.123548,0.427307,0.106375,2015020_C1_02.c3d
3,0.114621,0.113274,0.116819,0.116867,0.116216,0.121927,0.380911,0.123506,0.143970,0.125550,0.059930,0.076124,0.123065,0.475415,0.151562,2015020_C1_02.c3d
4,0.178806,0.176201,0.185391,0.186933,0.185193,0.178227,0.412715,0.179317,0.191243,0.180391,0.101186,0.122926,0.192042,0.496986,0.198848,2015020_C1_02.c3d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403451,0.047137,0.043727,0.067047,0.055559,0.051796,0.966873,0.975186,0.957168,0.983686,0.988087,0.002859,0.003795,0.008707,0.034307,0.029140,2014019_C3_05.c3d
403452,0.057407,0.054638,0.080028,0.069158,0.066056,0.955175,0.965014,0.946298,0.976387,0.981616,0.005643,0.006461,0.008198,0.030783,0.026065,2014019_C3_05.c3d
403453,0.031477,0.029549,0.055342,0.045675,0.043706,0.968980,0.978049,0.957591,0.986109,0.990714,0.002353,0.002770,0.004715,0.028024,0.023255,2014019_C3_05.c3d
403454,0.051332,0.050231,0.074529,0.066450,0.065715,0.949989,0.961823,0.940178,0.974401,0.980023,0.007282,0.007806,0.005375,0.024466,0.020370,2014019_C3_05.c3d


In [7]:
path = '../../Dataset/Processed/data1_f18_t6_n1_all_vel_p2.joblib'
data = {
    'X': features_new,
    'y': target
}
dump(data, path)

['../../Dataset/Processed/data1_f18_t6_n1_all_vel_p2.joblib']

# Generating Train Test Sets

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.model_selection import train_test_split

N_SAMPLES = 1024
N_CHANNELS = 15

# path = '../../Dataset/Processed/data2_f72_t3_n1_high_vel.joblib'
# data = load(path)
# features = data['X']
# target = data['y']

path1 = '../../Dataset/Processed/data1_f18_t6_n1_all_vel_p1.joblib'
path2 = '../../Dataset/Processed/data1_f18_t6_n1_all_vel_p2.joblib'

data1 = load(path1)
data2 = load(path2)

features1 = data1['X']
features2 = data2['X']

target1 = data1['y']
target2 = data2['y']

print(features1.shape)
print(features2.shape)

print(target1.shape)
print(target2.shape)

(1421312, 16)
(403456, 16)
(1421312, 6)
(403456, 6)


In [9]:
features = pd.concat([features1, features2])
target = pd.concat([target1, target2])

# print(features.shape)
# print(target.shape)

features

,fal_x,tam_x,fm5_x,fm1_x,fm2_x,fal_y,tam_y,fm5_y,fm1_y,fm2_y,fal_z,tam_z,fm5_z,fm1_z,fm2_z,filename
0,0.891303,0.891211,0.892550,0.891773,0.891070,0.109102,0.109443,0.107985,0.109229,0.109116,0.110040,0.109323,0.107434,0.099229,0.105993,2015027_C2_03.c3d
1,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2015027_C2_03.c3d
2,0.923510,0.923783,0.921982,0.923273,0.924111,0.077096,0.076543,0.076288,0.076798,0.077007,0.075654,0.076955,0.080820,0.083213,0.083141,2015027_C2_03.c3d
3,0.830596,0.831185,0.827719,0.830318,0.831874,0.171709,0.170493,0.169905,0.171047,0.171509,0.168563,0.171419,0.179939,0.185084,0.185018,2015027_C2_03.c3d
4,0.833045,0.833896,0.829157,0.832801,0.834872,0.171823,0.169897,0.169997,0.170808,0.171531,0.166841,0.171289,0.184421,0.196792,0.192435,2015027_C2_03.c3d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403451,0.047137,0.043727,0.067047,0.055559,0.051796,0.966873,0.975186,0.957168,0.983686,0.988087,0.002859,0.003795,0.008707,0.034307,0.029140,2014019_C3_05.c3d
403452,0.057407,0.054638,0.080028,0.069158,0.066056,0.955175,0.965014,0.946298,0.976387,0.981616,0.005643,0.006461,0.008198,0.030783,0.026065,2014019_C3_05.c3d
403453,0.031477,0.029549,0.055342,0.045675,0.043706,0.968980,0.978049,0.957591,0.986109,0.990714,0.002353,0.002770,0.004715,0.028024,0.023255,2014019_C3_05.c3d
403454,0.051332,0.050231,0.074529,0.066450,0.065715,0.949989,0.961823,0.940178,0.974401,0.980023,0.007282,0.007806,0.005375,0.024466,0.020370,2014019_C3_05.c3d


In [ ]:
# features_new = features.drop(columns=['fcc_x', 'fcc_y', 'fcc_z'])
# features_new = features_new[[
#     'fal_x', 'tam_x', 'fm5_x', 'fm1_x', 'fm2_x',
#     'fal_y', 'tam_y', 'fm5_y', 'fm1_y', 'fm2_y',
#     'fal_z', 'tam_z', 'fm5_z', 'fm1_z', 'fm2_z'
# ]]
# features_new

In [ ]:
x = features_new['fal_y'].to_numpy().reshape(-1, N_SAMPLES)
for i in range(x.shape[0]):
    plt.plot(x[i, :])

In [ ]:
# traget = target[['fx', 'fy', 'fz']]
traget

In [ ]:
X = features_new.to_numpy().reshape(-1, N_SAMPLES, N_CHANNELS)
y = target.to_numpy().reshape(-1, N_SAMPLES, 6)

print(X.shape)
print(y.shape)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

In [10]:
path = '../../Dataset/Processed/data1_f18_t6_n1_all_vel_combined.joblib'
# data = {
#     'train_X': train_X,
#     'test_X': test_X,
#     'train_y': train_y,
#     'test_y': test_y
# }
data = {
    'X': features,
    'y': target
}
dump(data, path)

['../../Dataset/Processed/data1_f18_t6_n1_all_vel_combined.joblib']

In [ ]:
for i in range(train_X.shape[2]):
    plt.plot(train_X[i, :, 2])